In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def scrape_league_data(league_name, url, table_id=None, season="2024/25"):
    print(f"Scraping {league_name}...")

    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers, verify=False)
    soup = BeautifulSoup(response.content, 'html.parser')

    table = soup.find("table", id=table_id) if table_id else soup.find("table")
    if not table:
        print(f"Table not found for {league_name}")
        return None

    df = pd.read_html(str(table))[0]
    df['League'] = league_name
    df['Season'] = season

    return df


In [ ]:
# List of leagues to scrape
years = ["2024-2025","2023-2024","2022-2023","2021-2022","2020-2021","2019-2020","2018-2019","2017-2018","2016-2017","2015-2016"]
leagueinfo = [
    {
        "name": "Bundesliga",
        "url": "https://fbref.com/en/comps/20/{year}/schedule/{year}-Bundesliga-Scores-and-Fixtures"

    },
    {
        "name": "Premier League",
        "url": "https://fbref.com/en/comps/9/{year}/schedule/{year}-Premier-League-Scores-and-Fixtures"
    },
    {
        "name": "Serie A",
        "url": "https://fbref.com/en/comps/11/{year}/schedule/{year}-Serie-A-Scores-and-Fixtures"
    }
    ,
    {
        "name": "La Liga",
        "url": "https://fbref.com/en/comps/12/{year}/schedule/{year}-La-Liga-Scores-and-Fixtures"
    },
    {
        "name": "Ligue 1",
        "url": "https://fbref.com/en/comps/13/{year}/schedule/{year}-Ligue-1-Scores-and-Fixtures"
    },
    {
        "name": "Eredivisie",
        "url": "https://fbref.com/en/comps/20/{year}/schedule/{year}-Eredivisie-Scores-and-Fixtures"
    },
    {
        "name": "Primeira Liga",
        "url": "https://fbref.com/en/comps/19/{year}/schedule/{year}-Primeira-Liga-Scores-and-Fixtures"
    },
    {
        "name": "EFL Championship",
        "url": "https://fbref.com/en/comps/10/{year}/schedule/{year}-EFL-Championship-Scores-and-Fixtures"
    }

]

leagues = []
for year in years:
    for league in leagueinfo:
        leagues.append({
            "name": f"{league['name']} {year}",
            "url": league["url"].format(year=year),
            "season": year.replace("-", "/")
        })

league_dfs = {}
combined_list = []

for league in leagues:
    df = scrape_league_data(league["name"], league["url"])
    if df is not None:
        league_dfs[league["name"]] = df 
        combined_list.append(df)        
